In [1]:
import torch
import torch.nn as nn
import ot
import numpy as np

2024-09-02 17:30:41.466880: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-02 17:30:46.290983: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def isnan(x):
    return x != x

In [3]:
class GroundMetric:
    """
        Ground Metric object for Wasserstein computations:

    """

    def __init__(self, params, not_squared = False):
        self.params = params
        self.ground_metric_type = params.ground_metric
        self.ground_metric_normalize = params.ground_metric_normalize
        self.reg = params.reg
        if hasattr(params, 'not_squared'):
            self.squared = not params.not_squared
        else:
            # so by default squared will be on!
            self.squared = not not_squared
        self.mem_eff = params.ground_metric_eff

    def _clip(self, ground_metric_matrix):
        if self.params.debug:
            print("before clipping", ground_metric_matrix.data)

        percent_clipped = (float((ground_metric_matrix >= self.reg * self.params.clip_max).long().sum().data) \
                           / ground_metric_matrix.numel()) * 100
        print("percent_clipped is (assumes clip_min = 0) ", percent_clipped)
        setattr(self.params, 'percent_clipped', percent_clipped)
        # will keep the M' = M/reg in range clip_min and clip_max
        ground_metric_matrix.clamp_(min=self.reg * self.params.clip_min,
                                             max=self.reg * self.params.clip_max)
        if self.params.debug:
            print("after clipping", ground_metric_matrix.data)
        return ground_metric_matrix

    def _normalize(self, ground_metric_matrix):

        if self.ground_metric_normalize == "log":
            ground_metric_matrix = torch.log1p(ground_metric_matrix)
        elif self.ground_metric_normalize == "max":
            print("Normalizing by max of ground metric and which is ", ground_metric_matrix.max())
            ground_metric_matrix = ground_metric_matrix / ground_metric_matrix.max()
        elif self.ground_metric_normalize == "median":
            print("Normalizing by median of ground metric and which is ", ground_metric_matrix.median())
            ground_metric_matrix = ground_metric_matrix / ground_metric_matrix.median()
        elif self.ground_metric_normalize == "mean":
            print("Normalizing by mean of ground metric and which is ", ground_metric_matrix.mean())
            ground_metric_matrix = ground_metric_matrix / ground_metric_matrix.mean()
        elif self.ground_metric_normalize == "none":
            return ground_metric_matrix
        else:
            raise NotImplementedError

        return ground_metric_matrix

    def _sanity_check(self, ground_metric_matrix):
        assert not (ground_metric_matrix < 0).any()
        assert not (isnan(ground_metric_matrix).any())

    def _cost_matrix_xy(self, x, y, p=2, squared = True):
        # TODO: Use this to guarantee reproducibility of previous results and then move onto better way
        "Returns the matrix of $|x_i-y_j|^p$."
        # (n_l,1,n_{l-1})
        x_col = x.unsqueeze(1)
        # (1, m_l, m_{l-1})
        # math induction -> n_{l-1} == m_{l-1}
        y_lin = y.unsqueeze(0)
        # c => (n_l, m_l, m_{l-1})
        c = torch.sum((torch.abs(x_col - y_lin)) ** p, 2)
        if not squared:
            print("dont leave off the squaring of the ground metric")
            c = c ** (1/2)
        # print(c.size())
        if self.params.dist_normalize:
            assert NotImplementedError
        return c


    def _pairwise_distances(self, x, y=None, squared=True):
        '''
        Source: https://discuss.pytorch.org/t/efficient-distance-matrix-computation/9065/2
        Input: x is a Nxd matrix
               y is an optional Mxd matirx
        Output: dist is a NxM matrix where dist[i,j] is the square norm between x[i,:] and y[j,:]
                if y is not given then use 'y=x'.
        i.e. dist[i,j] = ||x[i,:]-y[j,:]||^2
        '''
        x_norm = (x ** 2).sum(1).view(-1, 1)
        if y is not None:
            y_t = torch.transpose(y, 0, 1)
            y_norm = (y ** 2).sum(1).view(1, -1)
        else:
            y_t = torch.transpose(x, 0, 1)
            y_norm = x_norm.view(1, -1)

        dist = x_norm + y_norm - 2.0 * torch.mm(x, y_t)
        # Ensure diagonal is zero if x=y
        dist = torch.clamp(dist, min=0.0)

        if self.params.activation_histograms and self.params.dist_normalize:
            dist = dist/self.params.act_num_samples
            print("Divide squared distances by the num samples")

        if not squared:
            print("dont leave off the squaring of the ground metric")
            dist = dist ** (1/2)

        return dist

    def _get_euclidean(self, coordinates, other_coordinates=None):
        # TODO: Replace by torch.pdist (which is said to be much more memory efficient)

        if other_coordinates is None:
            matrix = torch.norm(
                coordinates.view(coordinates.shape[0], 1, coordinates.shape[1]) \
                - coordinates, p=2, dim=2
            )
        else:
            # memory efficient version
            if self.mem_eff:
                matrix = self._pairwise_distances(coordinates, other_coordinates, squared=self.squared)
            else:
                matrix = self._cost_matrix_xy(coordinates, other_coordinates, squared = self.squared)

        return matrix

    def _normed_vecs(self, vecs, eps=1e-9):
        norms = torch.norm(vecs, dim=-1, keepdim=True)
        print("stats of vecs are: mean {}, min {}, max {}, std {}".format(
            norms.mean(), norms.min(), norms.max(), norms.std()
        ))
        return vecs / (norms + eps)

    def _get_cosine(self, coordinates, other_coordinates=None):
        if other_coordinates is None:
            matrix = coordinates / torch.norm(coordinates, dim=1, keepdim=True)
            matrix = 1 - matrix @ matrix.t()
        else:
            matrix = 1 - torch.div(
                coordinates @ other_coordinates.t(),
                torch.norm(coordinates, dim=1).view(-1, 1) @ torch.norm(other_coordinates, dim=1).view(1, -1)
            )
        return matrix.clamp_(min=0)

    def _get_angular(self, coordinates, other_coordinates=None):
        pass

    def get_metric(self, coordinates, other_coordinates=None):
        get_metric_map = {
            'euclidean': self._get_euclidean,
            'cosine': self._get_cosine,
            'angular': self._get_angular,
        }
        return get_metric_map[self.ground_metric_type](coordinates, other_coordinates)

    def process(self, coordinates, other_coordinates=None):
        print('Processing the coordinates to form ground_metric')
        if self.params.geom_ensemble_type == 'wts' and self.params.normalize_wts:
            print("In weight mode: normalizing weights to unit norm")
            coordinates = self._normed_vecs(coordinates)
            if other_coordinates is not None:
                other_coordinates = self._normed_vecs(other_coordinates)

        ground_metric_matrix = self.get_metric(coordinates, other_coordinates)

        if self.params.debug:
            print("coordinates is ", coordinates)
            if other_coordinates is not None:
                print("other_coordinates is ", other_coordinates)
            print("ground_metric_matrix is ", ground_metric_matrix)

        self._sanity_check(ground_metric_matrix)

        ground_metric_matrix = self._normalize(ground_metric_matrix)

        self._sanity_check(ground_metric_matrix)

        if self.params.clip_gm:
            ground_metric_matrix = self._clip(ground_metric_matrix)

        self._sanity_check(ground_metric_matrix)

        if self.params.debug:
            print("ground_metric_matrix at the end is ", ground_metric_matrix)

        return ground_metric_matrix


In [4]:
def cost_matrix(x, y, p=2):
    "Returns the matrix of $|x_i-y_j|^p$."
    x_col = x.unsqueeze(1)
    y_lin = y.unsqueeze(0)
    c = torch.sum((torch.abs(x_col - y_lin)) ** p, 2)
    return c

def get_histogram(args, idx, cardinality, layer_name, activations=None, return_numpy = True, float64=False):
    if activations is None:
        # returns a uniform measure
        if not args.unbalanced:
            print("returns a uniform measure of cardinality: ", cardinality)
            return np.ones(cardinality)/cardinality
        else:
            return np.ones(cardinality)
    else:
        # return softmax over the activations raised to a temperature
        # layer_name is like 'fc1.weight', while activations only contains 'fc1'
        print(activations[idx].keys())
        unnormalized_weights = activations[idx][layer_name.split('.')[0]]
        print("For layer {},  shape of unnormalized weights is ".format(layer_name), unnormalized_weights.shape)
        unnormalized_weights = unnormalized_weights.squeeze()
        assert unnormalized_weights.shape[0] == cardinality

        if return_numpy:
            if float64:
                return torch.softmax(unnormalized_weights / args.softmax_temperature, dim=0).data.cpu().numpy().astype(
                    np.float64)
            else:
                return torch.softmax(unnormalized_weights / args.softmax_temperature, dim=0).data.cpu().numpy()
        else:
            return torch.softmax(unnormalized_weights / args.softmax_temperature, dim=0)

In [5]:
def get_wassersteinized_layers_modularized(args, networks, activations=None, eps=1e-7, test_loader=None):
    '''
    Two neural networks that have to be averaged in geometric manner (i.e. layerwise).

    The 1st network is aligned with respect to the other via wasserstein distance.
    Also this assumes that all the layers are either fully connected or convolutional *(with no bias)*

    :param networks: list of networks
    :param activations: If not None, use it to build the activation histograms.
    Otherwise assumes uniform distribution over neurons in a layer.
    :return: list of layer weights 'wassersteinized'
    '''

    # simple_model_0, simple_model_1 = networks[0], networks[1]
    # simple_model_0 = get_trained_model(0, model='simplenet')
    # simple_model_1 = get_trained_model(1, model='simplenet')

    avg_aligned_layers = []
    # cumulative_T_var = None
    T_var = None
    # print(list(networks[0].parameters()))
    previous_layer_shape = None
    ground_metric_object = GroundMetric(args)

    if args.eval_aligned:
        model0_aligned_layers = []

    if args.gpu_id==-1:
        device = torch.device('cpu')
    else:
        device = torch.device('cuda:{}'.format(args.gpu_id))


    num_layers = len(list(zip(networks[0].parameters(), networks[1].parameters())))
    # named_parameters is an generator
    # for linear layer with bias, the affine matrix and bias will be two separate terms in named_parameters
    for idx, ((layer0_name, fc_layer0_weight), (layer1_name, fc_layer1_weight)) in \
            enumerate(zip(networks[0].named_parameters(), networks[1].named_parameters())):

        assert fc_layer0_weight.shape == fc_layer1_weight.shape
        print("Previous layer shape is ", previous_layer_shape)
        previous_layer_shape = fc_layer1_weight.shape

        mu_cardinality = fc_layer0_weight.shape[0]
        nu_cardinality = fc_layer1_weight.shape[0]

        # mu = np.ones(fc_layer0_weight.shape[0])/fc_layer0_weight.shape[0]
        # nu = np.ones(fc_layer1_weight.shape[0])/fc_layer1_weight.shape[0]

        layer_shape = fc_layer0_weight.shape
        if len(layer_shape) > 2:
            is_conv = True
            # For convolutional layers, it is (#out_channels, #in_channels, height, width)
            fc_layer0_weight_data = fc_layer0_weight.data.view(fc_layer0_weight.shape[0], fc_layer0_weight.shape[1], -1)
            fc_layer1_weight_data = fc_layer1_weight.data.view(fc_layer1_weight.shape[0], fc_layer1_weight.shape[1], -1)
        else:
            is_conv = False
            fc_layer0_weight_data = fc_layer0_weight.data
            fc_layer1_weight_data = fc_layer1_weight.data

        if idx == 0:
            if is_conv:
                M = ground_metric_object.process(fc_layer0_weight_data.view(fc_layer0_weight_data.shape[0], -1),
                                fc_layer1_weight_data.view(fc_layer1_weight_data.shape[0], -1))
                # M = cost_matrix(fc_layer0_weight_data.view(fc_layer0_weight_data.shape[0], -1),
                #                 fc_layer1_weight_data.view(fc_layer1_weight_data.shape[0], -1))
            else:
                # print("layer data is ", fc_layer0_weight_data, fc_layer1_weight_data)
                M = ground_metric_object.process(fc_layer0_weight_data, fc_layer1_weight_data)
                # M = cost_matrix(fc_layer0_weight, fc_layer1_weight)

            aligned_wt = fc_layer0_weight_data
        else:

            print("shape of layer: model 0", fc_layer0_weight_data.shape)
            print("shape of layer: model 1", fc_layer1_weight_data.shape)
            print("shape of previous transport map", T_var.shape)

            # aligned_wt = None, this caches the tensor and causes OOM
            if is_conv:
                T_var_conv = T_var.unsqueeze(0).repeat(fc_layer0_weight_data.shape[2], 1, 1)
                aligned_wt = torch.bmm(fc_layer0_weight_data.permute(2, 0, 1), T_var_conv).permute(1, 2, 0)

                M = ground_metric_object.process(
                    aligned_wt.contiguous().view(aligned_wt.shape[0], -1),
                    fc_layer1_weight_data.view(fc_layer1_weight_data.shape[0], -1)
                )
            else:
                # ? why 
                if fc_layer0_weight.data.shape[1] != T_var.shape[0]:
                    # Handles the switch from convolutional layers to fc layers
                    fc_layer0_unflattened = fc_layer0_weight.data.view(fc_layer0_weight.shape[0], T_var.shape[0], -1).permute(2, 0, 1)
                    aligned_wt = torch.bmm(
                        fc_layer0_unflattened,
                        T_var.unsqueeze(0).repeat(fc_layer0_unflattened.shape[0], 1, 1)
                    ).permute(1, 2, 0)
                    aligned_wt = aligned_wt.contiguous().view(aligned_wt.shape[0], -1)
                else:
                    # print("layer data (aligned) is ", aligned_wt, fc_layer1_weight_data)
                    aligned_wt = torch.matmul(fc_layer0_weight.data, T_var)
                # M = cost_matrix(aligned_wt, fc_layer1_weight)
                M = ground_metric_object.process(aligned_wt, fc_layer1_weight)
            #     print("ground metric is ", M)
            # if args.skip_last_layer and idx == (num_layers - 1):
            #     print("Simple averaging of last layer weights. NO transport map needs to be computed")
            #     if args.ensemble_step != 0.5:
            #         avg_aligned_layers.append((1 - args.ensemble_step) * aligned_wt +
            #                               args.ensemble_step * fc_layer1_weight)
            #     else:
            #         avg_aligned_layers.append((aligned_wt + fc_layer1_weight)/2)
            #     return avg_aligned_layers

        if args.importance is None or (idx == num_layers -1):
            mu = get_histogram(args, 0, mu_cardinality, layer0_name)
            nu = get_histogram(args, 1, nu_cardinality, layer1_name)
        # else:
        #     # mu = _get_neuron_importance_histogram(args, aligned_wt, is_conv)
        #     mu = _get_neuron_importance_histogram(args, fc_layer0_weight_data, is_conv)
        #     nu = _get_neuron_importance_histogram(args, fc_layer1_weight_data, is_conv)
        #     print(mu, nu)
        #     assert args.proper_marginals

        cpuM = M.data.cpu().numpy()
        if args.exact:
            T = ot.emd(mu, nu, cpuM)
        else:
            T = ot.bregman.sinkhorn(mu, nu, cpuM, reg=args.reg)
        # T = ot.emd(mu, nu, log_cpuM)

        if args.gpu_id!=-1:
            T_var = torch.from_numpy(T).cuda(args.gpu_id).float()
        else:
            T_var = torch.from_numpy(T).float()

        # torch.set_printoptions(profile="full")
        print("the transport map is ", T_var)
        # torch.set_printoptions(profile="default")

        if args.correction:
            if not args.proper_marginals:
                # think of it as m x 1, scaling weights for m linear combinations of points in X
                if args.gpu_id != -1:
                    # marginals = torch.mv(T_var.t(), torch.ones(T_var.shape[0]).cuda(args.gpu_id))  # T.t().shape[1] = T.shape[0]
                    marginals = torch.ones(T_var.shape[0]).cuda(args.gpu_id) / T_var.shape[0]
                else:
                    # marginals = torch.mv(T_var.t(),
                    #                      torch.ones(T_var.shape[0]))  # T.t().shape[1] = T.shape[0]
                    marginals = torch.ones(T_var.shape[0]) / T_var.shape[0]
                marginals = torch.diag(1.0/(marginals + eps))  # take inverse
                T_var = torch.matmul(T_var, marginals)
            else:
                # marginals_alpha = T_var @ torch.ones(T_var.shape[1], dtype=T_var.dtype).to(device)
                marginals_beta = T_var.t() @ torch.ones(T_var.shape[0], dtype=T_var.dtype).to(device)

                marginals = (1 / (marginals_beta + eps))
                print("shape of inverse marginals beta is ", marginals_beta.shape)
                print("inverse marginals beta is ", marginals_beta)

                T_var = T_var * marginals
                # i.e., how a neuron of 2nd model is constituted by the neurons of 1st model
                # this should all be ones, and number equal to number of neurons in 2nd model
                print(T_var.sum(dim=0))
                # assert (T_var.sum(dim=0) == torch.ones(T_var.shape[1], dtype=T_var.dtype).to(device)).all()

        if args.debug:
            if idx == (num_layers - 1):
                print("there goes the last transport map: \n ", T_var)
            else:
                print("there goes the transport map at layer {}: \n ".format(idx), T_var)

            print("Ratio of trace to the matrix sum: ", torch.trace(T_var) / torch.sum(T_var))

        print("Ratio of trace to the matrix sum: ", torch.trace(T_var) / torch.sum(T_var))
        print("Here, trace is {} and matrix sum is {} ".format(torch.trace(T_var), torch.sum(T_var)))
        setattr(args, 'trace_sum_ratio_{}'.format(layer0_name), (torch.trace(T_var) / torch.sum(T_var)).item())

        if args.past_correction:
            print("this is past correction for weight mode")
            print("Shape of aligned wt is ", aligned_wt.shape)
            print("Shape of fc_layer0_weight_data is ", fc_layer0_weight_data.shape)
            t_fc0_model = torch.matmul(T_var.t(), aligned_wt.contiguous().view(aligned_wt.shape[0], -1))
        else:
            t_fc0_model = torch.matmul(T_var.t(), fc_layer0_weight_data.view(fc_layer0_weight_data.shape[0], -1))

        # Average the weights of aligned first layers
        if args.ensemble_step != 0.5:
            geometric_fc = ((1-args.ensemble_step) * t_fc0_model +
                            args.ensemble_step * fc_layer1_weight_data.view(fc_layer1_weight_data.shape[0], -1))
        else:
            geometric_fc = (t_fc0_model + fc_layer1_weight_data.view(fc_layer1_weight_data.shape[0], -1))/2
        if is_conv and layer_shape != geometric_fc.shape:
            geometric_fc = geometric_fc.view(layer_shape)
        avg_aligned_layers.append(geometric_fc)

        # # get the performance of the model 0 aligned with respect to the model 1
        # if args.eval_aligned:
        #     if is_conv and layer_shape != t_fc0_model.shape:
        #         t_fc0_model = t_fc0_model.view(layer_shape)
        #     model0_aligned_layers.append(t_fc0_model)
        #     _, acc = update_model(args, networks[0], model0_aligned_layers, test=True,
        #                           test_loader=test_loader, idx=0)
        #     print("For layer idx {}, accuracy of the updated model is {}".format(idx, acc))
        #     setattr(args, 'model0_aligned_acc_layer_{}'.format(str(idx)), acc)
        #     if idx == (num_layers - 1):
        #         setattr(args, 'model0_aligned_acc', acc)

    return avg_aligned_layers

In [56]:
from typing import List, Callable, Union, Any, TypeVar, Tuple
# from torch import tensor as Tensor

Tensor = TypeVar('torch.tensor')

from abc import abstractmethod

class BaseVAE(nn.Module):
    
    def __init__(self) -> None:
        super(BaseVAE, self).__init__()

    def encode(self, input: Tensor) -> List[Tensor]:
        raise NotImplementedError

    def decode(self, input: Tensor) -> Any:
        raise NotImplementedError

    def sample(self, batch_size:int, current_device: int, **kwargs) -> Tensor:
        raise NotImplementedError

    def generate(self, x: Tensor, **kwargs) -> Tensor:
        raise NotImplementedError

    @abstractmethod
    def forward(self, *inputs: Tensor) -> Tensor:
        pass

    @abstractmethod
    def loss_function(self, *inputs: Any, **kwargs) -> Tensor:
        pass


class VanillaVAE(nn.Module):


    def __init__(self,
                 in_channels: int,
                 latent_dim: int,
                 hidden_dims: List = None,
                 **kwargs) -> None:
        super(VanillaVAE, self).__init__()

        self.latent_dim = latent_dim

        modules = []
        if hidden_dims is None:
            hidden_dims = [32, 64, 128, 256, 512]

        # Build Encoder
        for h_dim in hidden_dims:
            modules.append(
                nn.Sequential(
                    nn.Conv2d(in_channels, out_channels=h_dim,
                              kernel_size= 3, stride= 2, padding  = 1),
                    nn.BatchNorm2d(h_dim),
                    nn.LeakyReLU())
            )
            in_channels = h_dim

        self.encoder = nn.Sequential(*modules)
        self.fc_mu = nn.Linear(hidden_dims[-1]*4, latent_dim)
        self.fc_var = nn.Linear(hidden_dims[-1]*4, latent_dim)


        # Build Decoder
        modules = []

        self.decoder_input = nn.Linear(latent_dim, hidden_dims[-1] * 4)

        hidden_dims.reverse()

        for i in range(len(hidden_dims) - 1):
            modules.append(
                nn.Sequential(
                    nn.ConvTranspose2d(hidden_dims[i],
                                       hidden_dims[i + 1],
                                       kernel_size=3,
                                       stride = 2,
                                       padding=1,
                                       output_padding=1),
                    nn.BatchNorm2d(hidden_dims[i + 1]),
                    nn.LeakyReLU())
            )



        self.decoder = nn.Sequential(*modules)

        self.final_layer = nn.Sequential(
                            nn.ConvTranspose2d(hidden_dims[-1],
                                               hidden_dims[-1],
                                               kernel_size=3,
                                               stride=2,
                                               padding=1,
                                               output_padding=1),
                            nn.BatchNorm2d(hidden_dims[-1]),
                            nn.LeakyReLU(),
                            nn.Conv2d(hidden_dims[-1], out_channels= 3,kernel_size= 3, padding= 1),
                            nn.Tanh())

    def encode(self, input: Tensor) -> List[Tensor]:
        """
        Encodes the input by passing through the encoder network
        and returns the latent codes.
        :param input: (Tensor) Input tensor to encoder [N x C x H x W]
        :return: (Tensor) List of latent codes
        """
        result = self.encoder(input)
        # print("After encoder: ", result.shape)
        result = torch.flatten(result, start_dim=1)
        # print("After encoder and flatten: ", result.shape)

        # Split the result into mu and var components
        # of the latent Gaussian distribution
        mu = self.fc_mu(result)
        log_var = self.fc_var(result)

        # print("mu:", mu.shape)
        # print("log_var;", log_var.shape)

        return [mu, log_var]

    def decode(self, z: Tensor) -> Tensor:
        """
        Maps the given latent codes
        onto the image space.
        :param z: (Tensor) [B x D]
        :return: (Tensor) [B x C x H x W]
        """
        result = self.decoder_input(z)
        # print("after decoder_input layer:", result.shape)
        result = result.view(-1, 512, 2, 2)
        # print("after reshape:", result.shape)
        result = self.decoder(result)
        # print("after decoder:", result.shape)
        result = self.final_layer(result)
        # print("after final layer:", result.shape)
        return result

    def reparameterize(self, mu: Tensor, logvar: Tensor) -> Tensor:
        """
        Reparameterization trick to sample from N(mu, var) from
        N(0,1).
        :param mu: (Tensor) Mean of the latent Gaussian [B x D]
        :param logvar: (Tensor) Standard deviation of the latent Gaussian [B x D]
        :return: (Tensor) [B x D]
        """
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return eps * std + mu

    def forward(self, input: Tensor, **kwargs) -> List[Tensor]:
        mu, log_var = self.encode(input)
        z = self.reparameterize(mu, log_var)
        return  [self.decode(z), input, mu, log_var]

    def loss_function(self,
                      *args,
                      **kwargs) -> dict:
        """
        Computes the VAE loss function.
        KL(N(\mu, \sigma), N(0, 1)) = \log \frac{1}{\sigma} + \frac{\sigma^2 + \mu^2}{2} - \frac{1}{2}
        :param args:
        :param kwargs:
        :return:
        """
        recons = args[0]
        input = args[1]
        mu = args[2]
        log_var = args[3]

        kld_weight = kwargs['M_N'] # Account for the minibatch samples from the dataset
        recons_loss =F.mse_loss(recons, input)


        kld_loss = torch.mean(-0.5 * torch.sum(1 + log_var - mu ** 2 - log_var.exp(), dim = 1), dim = 0)

        loss = recons_loss + kld_weight * kld_loss
        return {'loss': loss, 'Reconstruction_Loss':recons_loss.detach(), 'KLD':-kld_loss.detach()}

    def sample(self,
               num_samples:int,
               current_device: int, **kwargs) -> Tensor:
        """
        Samples from the latent space and return the corresponding
        image space map.
        :param num_samples: (Int) Number of samples
        :param current_device: (Int) Device to run the model
        :return: (Tensor)
        """
        z = torch.randn(num_samples,
                        self.latent_dim)

        z = z.to(current_device)

        samples = self.decode(z)
        return samples

    def generate(self, x: Tensor, **kwargs) -> Tensor:
        """
        Given an input image x, returns the reconstructed image
        :param x: (Tensor) [B x C x H x W]
        :return: (Tensor) [B x C x H x W]
        """

        return self.forward(x)[0]

In [31]:
model1 = VanillaVAE(3, 128)
model2 = VanillaVAE(3, 128)
dict1 = dict(torch.load('vae_nob_bias_adagrad_1.ckpt', map_location=torch.device('cpu'))['state_dict'])
dict2 = dict(torch.load('vae_nob_bias_adagrad_2.ckpt', map_location=torch.device('cpu'))['state_dict'])
tmp1 = dict()
tmp2 = dict()

# print(dict1.keys())
for k in dict1.keys():
    v1 = dict1[k]
    v2 = dict2[k]
    k = k[6:]
    # print(k)
    # if "running" in k:
    #     print(v1 - v2)
    tmp1[k] = v1
    tmp2[k] = v2

# for k in dict2.keys():
#     v = dict2[k]
#     k = k[6:]
#     # print(k)
#     tmp2[k] = v

model1.load_state_dict(tmp1)
model2.load_state_dict(tmp2)

<All keys matched successfully>

In [ ]:
model1, model2

(VanillaVAE(
   (encoder): Sequential(
     (0): Sequential(
       (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
       (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): LeakyReLU(negative_slope=0.01)
     )
     (1): Sequential(
       (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
       (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): LeakyReLU(negative_slope=0.01)
     )
     (2): Sequential(
       (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
       (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): LeakyReLU(negative_slope=0.01)
     )
     (3): Sequential(
       (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
       (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): LeakyReLU(negative_slope=0.

In [ ]:
from parameters import get_parameters

args = get_parameters()

get_wassersteinized_layers_modularized(args, [model1, model2])

ModuleNotFoundError: No module named 'vgg'

In [27]:
from torchvision import transforms
from torchvision.datasets import CelebA
import torch.nn.functional as F

In [18]:
train_transforms = transforms.Compose([transforms.RandomHorizontalFlip(),
                                        transforms.CenterCrop(148),
                                        transforms.Resize(64),
                                        transforms.ToTensor(),])

In [32]:
test_data = CelebA(root='/home/chenyuheng/celeba', split='test', download=False, transform=train_transforms) 

In [33]:
def loaddata():
    return torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)   

from tqdm import tqdm
def test(model):
    model.cuda()
    dataloader = loaddata()
    i = 0
    loss = 0.0
    recon_loss = 0.0
    kl_loss = 0.0
    for img, _ in tqdm(dataloader):
        # print(img)
        out = model(img.cuda())
        # loss = loss * (i / (i + 1)) + model.loss_function(*out, M_N=0.00025)['loss'].item() / (i + 1)
        loss = loss * (i / (i + 1)) + model.loss_function(*out, M_N=0.00025)['loss'].item() / (i + 1)
        recon_loss = recon_loss * (i / (i + 1)) + model.loss_function(*out, M_N=0.00025)['Reconstruction_Loss'].item() / (i + 1)
        kl_loss = kl_loss * (i / (i + 1)) + model.loss_function(*out, M_N=0.00025)['KLD'].item() / (i + 1)
        # loss = loss * (i / (i + 1)) + model.loss_function(*out, M_N=0.00025)['loss'].item() / (64 * (i + 1))
        i += 1
    return {'loss': loss, 'recon_loss': recon_loss, 'kl_loss': kl_loss}

def get_avg_model(t):
    avg_model = VanillaVAE(3, 128)
    avg_tmp = dict()
    for k in dict1.keys():
        v1 = dict1[k]
        v2 = dict2[k]
        k = k[6:]
        avg_tmp[k] = t * v1 + (1 - t) * v2
    avg_model.load_state_dict(avg_tmp)
    return avg_model

for t in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    print(test(get_avg_model(t)))
# print(test(fused_model))
print(test(model1), test(model2))

100%|██████████| 312/312 [00:33<00:00,  9.28it/s]


{'loss': 0.020665558407274194, 'recon_loss': 0.014973822381132509, 'kl_loss': -22.76694299013185}


100%|██████████| 312/312 [00:29<00:00, 10.73it/s]


{'loss': 0.020688911350682758, 'recon_loss': 0.014985132356508623, 'kl_loss': -22.815114779350075}


100%|██████████| 312/312 [00:29<00:00, 10.71it/s]


...

{'loss': 0.020625482689445977, 'recon_loss': 0.014851278795574147, 'kl_loss': -23.096814357317413}


100%|██████████| 312/312 [00:31<00:00,  9.80it/s]


{'loss': 0.020577109753130325, 'recon_loss': 0.014786915516313651, 'kl_loss': -23.16077612607907}


100%|██████████| 312/312 [00:29<00:00, 10.55it/s]

{'loss': 0.020502921534129057, 'recon_loss': 0.014692730876282813, 'kl_loss': -23.24076166519752}


In [30]:
test(model1)

100%|██████████| 312/312 [00:30<00:00, 10.22it/s]


{'loss': 0.020466407510237063,
 'recon_loss': 0.014629446235724179,
 'kl_loss': -23.34784392821486}

In [34]:
test(model2)

100%|██████████| 312/312 [00:30<00:00, 10.22it/s]


{'loss': 0.02069165680008249,
 'recon_loss': 0.015014265783322168,
 'kl_loss': -22.709562717339914}

In [37]:
print(model1, model2)

VanillaVAE(
  (encoder): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
    )
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
    )
    (3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
    )
    (4): Sequential(
      (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
    )
  )
  (fc_mu): Linear(in_features=2048, out_features=128, bias=True)
  (fc_var): Linear(in_features=2048, out_features=128, bias=True)
  (decoder_input): Linear(in_features=128, out_features=2048, bias=True)
  (decoder): Sequenti

In [42]:
param_diff = 0.0
for param1, param2 in zip(model1.parameters(), model2.parameters()):
    print(param1.shape, param2.shape)
    param_diff += torch.square(param1.cpu() - param2.cpu()).sum()

torch.Size([32, 3, 3, 3]) torch.Size([32, 3, 3, 3])
torch.Size([32]) torch.Size([32])
torch.Size([64, 32, 3, 3]) torch.Size([64, 32, 3, 3])
torch.Size([64]) torch.Size([64])
torch.Size([128, 64, 3, 3]) torch.Size([128, 64, 3, 3])
torch.Size([128]) torch.Size([128])
torch.Size([256, 128, 3, 3]) torch.Size([256, 128, 3, 3])
torch.Size([256]) torch.Size([256])
torch.Size([512, 256, 3, 3]) torch.Size([512, 256, 3, 3])
torch.Size([512]) torch.Size([512])
torch.Size([128, 2048]) torch.Size([128, 2048])
torch.Size([128]) torch.Size([128])
torch.Size([128, 2048]) torch.Size([128, 2048])
torch.Size([128]) torch.Size([128])
torch.Size([2048, 128]) torch.Size([2048, 128])
torch.Size([2048]) torch.Size([2048])
torch.Size([512, 256, 3, 3]) torch.Size([512, 256, 3, 3])
torch.Size([256]) torch.Size([256])
torch.Size([256, 128, 3, 3]) torch.Size([256, 128, 3, 3])
torch.Size([128]) torch.Size([128])
torch.Size([128, 64, 3, 3]) torch.Size([128, 64, 3, 3])
torch.Size([64]) torch.Size([64])
torch.Size([64

In [43]:
param_diff

tensor(253.8647, grad_fn=<AddBackward0>)

In [57]:
model3 = VanillaVAE(3, 128)
model4 = VanillaVAE(3, 128)

dict3 = dict(torch.load('last7.ckpt', map_location=torch.device('cpu'))['state_dict'])
dict4 = dict(torch.load('last8.ckpt', map_location=torch.device('cpu'))['state_dict'])
tmp3 = dict()
tmp4 = dict()

# print(dict1.keys())
for k in dict3.keys():
    v1 = dict3[k]
    v2 = dict4[k]
    k = k[6:]
    # print(k)
    # if "running" in k:
    #     print(v1 - v2)
    tmp3[k] = v1
    tmp4[k] = v2

# for k in dict2.keys():
#     v = dict2[k]
#     k = k[6:]
#     # print(k)
#     tmp2[k] = v

model3.load_state_dict(tmp3)
model4.load_state_dict(tmp4)

<All keys matched successfully>

In [58]:
param_diff = 0.0
for param1, param2 in zip(model3.parameters(), model4.parameters()):
    # print(param1.shape, param2.shape)
    param_diff += torch.square(param1.cpu() - param2.cpu()).sum()
param_diff

tensor(1775210., grad_fn=<AddBackward0>)

In [40]:
lst1 = [1, 2, 3]
lst2 = [4, 5, 6]
for i,j in zip(lst1, lst2):
    print(i, j)

1 4
2 5
3 6


In [19]:
import PIL
# import PIL.Image
img = PIL.Image.open("origin_VanillaVAE_Epoch_0.png")
train_transforms(img).shape

torch.Size([3, 64, 64])

In [ ]:
model1(train_transforms(img).unsqueeze(0))
# model1(train_transforms(img).unsqueeze(0))

After encoder:  torch.Size([1, 512, 2, 2])
After encoder and flatten:  torch.Size([1, 2048])
mu: torch.Size([1, 128])
log_var; torch.Size([1, 128])
after decoder_input layer: torch.Size([1, 2048])
after reshape: torch.Size([1, 512, 2, 2])
after decoder: torch.Size([1, 32, 32, 32])
after final layer: torch.Size([1, 3, 64, 64])


[tensor([[[[ 0.2730,  0.2414,  0.3393,  ...,  0.0675,  0.0332,  0.0380],
           [ 0.2811,  0.2200,  0.2551,  ..., -0.0875, -0.1410, -0.0063],
           [ 0.3443,  0.2530,  0.3116,  ..., -0.1292, -0.1672,  0.0112],
           ...,
           [ 0.4955,  0.3060,  0.2658,  ..., -0.0529, -0.1468, -0.0974],
           [ 0.4762,  0.1809,  0.2290,  ..., -0.1909, -0.1936, -0.0057],
           [ 0.5777,  0.3117,  0.3108,  ..., -0.0562, -0.0455,  0.0692]],
 
          [[ 0.3769,  0.3818,  0.4219,  ...,  0.1275,  0.0876,  0.0785],
           [ 0.4299,  0.4136,  0.4111,  ...,  0.0797,  0.0208,  0.0655],
           [ 0.4577,  0.4050,  0.4144,  ...,  0.0268, -0.0462,  0.0297],
           ...,
           [ 0.6456,  0.5882,  0.5461,  ...,  0.1249,  0.0311, -0.0054],
           [ 0.6297,  0.4919,  0.4908,  ...,  0.0103,  0.0086,  0.0778],
           [ 0.6626,  0.5236,  0.4855,  ...,  0.1008,  0.0734,  0.0894]],
 
          [[ 0.4331,  0.4576,  0.5088,  ...,  0.1222,  0.1477,  0.1713],
           [ 

In [ ]:
t = Tensor(1, 3, 64, 64)
t

In [2]:
from vaes.vanilla_vae import VanillaVAE

model = VanillaVAE(3, 64)
symbolic_traced : torch.fx.GraphModule = torch.fx.symbolic_trace(model)
print(symbolic_traced.graph)
symbolic_traced.graph.print_tabular()

Initialize VanillaVAE track_running_stats:False bias:True
graph():
    %input_1 : ~torch.tensor [#users=2] = placeholder[target=input]
    %_kwargs : [#users=0] = placeholder[target=**kwargs]
    %encoder_0_0 : [#users=1] = call_module[target=encoder.0.0](args = (%input_1,), kwargs = {})
    %encoder_0_1 : [#users=1] = call_module[target=encoder.0.1](args = (%encoder_0_0,), kwargs = {})
    %encoder_0_2 : [#users=1] = call_module[target=encoder.0.2](args = (%encoder_0_1,), kwargs = {})
    %encoder_1_0 : [#users=1] = call_module[target=encoder.1.0](args = (%encoder_0_2,), kwargs = {})
    %encoder_1_1 : [#users=1] = call_module[target=encoder.1.1](args = (%encoder_1_0,), kwargs = {})
    %encoder_1_2 : [#users=1] = call_module[target=encoder.1.2](args = (%encoder_1_1,), kwargs = {})
    %encoder_2_0 : [#users=1] = call_module[target=encoder.2.0](args = (%encoder_1_2,), kwargs = {})
    %encoder_2_1 : [#users=1] = call_module[target=encoder.2.1](args = (%encoder_2_0,), kwargs = {})
    

In [16]:
for node in symbolic_traced.graph.nodes:
    # print(node.op, node.is_impure())
    # print(type(node.args))
    # print(len(node.args))
    if len(node.args) != 0:
        print(type(node.args[0]), type(node.op), type(node.target))

<class 'torch.fx.node.Node'> <class 'str'> <class 'str'>
<class 'torch.fx.node.Node'> <class 'str'> <class 'str'>
<class 'torch.fx.node.Node'> <class 'str'> <class 'str'>
<class 'torch.fx.node.Node'> <class 'str'> <class 'str'>
<class 'torch.fx.node.Node'> <class 'str'> <class 'str'>
<class 'torch.fx.node.Node'> <class 'str'> <class 'str'>
<class 'torch.fx.node.Node'> <class 'str'> <class 'str'>
<class 'torch.fx.node.Node'> <class 'str'> <class 'str'>
<class 'torch.fx.node.Node'> <class 'str'> <class 'str'>
<class 'torch.fx.node.Node'> <class 'str'> <class 'str'>
<class 'torch.fx.node.Node'> <class 'str'> <class 'str'>
<class 'torch.fx.node.Node'> <class 'str'> <class 'str'>
<class 'torch.fx.node.Node'> <class 'str'> <class 'str'>
<class 'torch.fx.node.Node'> <class 'str'> <class 'str'>
<class 'torch.fx.node.Node'> <class 'str'> <class 'str'>
<class 'torch.fx.node.Node'> <class 'str'> <class 'str'>
<class 'torch.fx.node.Node'> <class 'str'> <class 'str'>
<class 'torch.fx.node.Node'> <c

In [17]:
def get_model_structure(model):
    gm = torch.fx.symbolic_trace(model)
    graph = gm.graph
    node_list = graph.nodes
    gm.graph.print_tabular()
    tmp = dict()
    for node in node_list:
        print(node.name)
        if node.op == 'call_module':
            tmp[node.target] = []
            for prev in node.all_input_nodes:
                if prev.op == 'call_module':
                    tmp[node.target].extend([prev.target])
                else:
                    tmp[node.target].extend(tmp[prev.name])
        else:
            tmp[node.name] = []
            for prev in node.all_input_nodes:
                if prev.op == 'call module':
                    tmp[node.name].extend(tmp[prev.target])
                else:
                    tmp[node.name].extend(tmp[prev.name])
    return tmp

In [23]:
def get_model_structure(model):
    gm = torch.fx.symbolic_trace(model.cuda())
    graph = gm.graph
    node_list = graph.nodes
    gm.graph.print_tabular()
    tmp = dict()
    for node in node_list:
        print(node.name)
        if node.op == 'call_module':
            tmp[node.target] = set()
            for prev in node.all_input_nodes:
                if prev.op == 'call_module':
                    tmp[node.target].add(prev.target)
                else:
                    # tmp[node.target].extend(tmp[prev.name])
                    tmp[node.target] |= tmp[prev.name]

        else:
            tmp[node.name] = set()
            for prev in node.all_input_nodes:
                if prev.op == 'call_module':
                    # tmp[node.name].extend(tmp[prev.target])
                    tmp[node.name].add(prev.target)
                else:
                    # tmp[node.name].extend(tmp[prev.name])
                    tmp[node.name] |= tmp[prev.name]

    return tmp

In [24]:
get_model_structure(model)

opcode         name           target                                                         args                                        kwargs
-------------  -------------  -------------------------------------------------------------  ------------------------------------------  ----------------
placeholder    input_1        input                                                          ()                                          {}
placeholder    _kwargs        **kwargs                                                       ()                                          {}
call_module    encoder_0_0    encoder.0.0                                                    (input_1,)                                  {}
call_module    encoder_0_1    encoder.0.1                                                    (encoder_0_0,)                              {}
call_module    encoder_0_2    encoder.0.2                                                    (encoder_0_1,)                              {}
ca

{'input_1': set(),
 '_kwargs': set(),
 'encoder.0.0': set(),
 'encoder.0.1': {'encoder.0.0'},
 'encoder.0.2': {'encoder.0.1'},
 'encoder.1.0': {'encoder.0.2'},
 'encoder.1.1': {'encoder.1.0'},
 'encoder.1.2': {'encoder.1.1'},
 'encoder.2.0': {'encoder.1.2'},
 'encoder.2.1': {'encoder.2.0'},
 'encoder.2.2': {'encoder.2.1'},
 'encoder.3.0': {'encoder.2.2'},
 'encoder.3.1': {'encoder.3.0'},
 'encoder.3.2': {'encoder.3.1'},
 'encoder.4.0': {'encoder.3.2'},
 'encoder.4.1': {'encoder.4.0'},
 'encoder.4.2': {'encoder.4.1'},
 'encoder.5.0': {'encoder.4.2'},
 'encoder.5.1': {'encoder.5.0'},
 'encoder.5.2': {'encoder.5.1'},
 'flatten': {'encoder.5.2'},
 'fc_mu': {'encoder.5.2'},
 'fc_var': {'encoder.5.2'},
 'mul': {'fc_var'},
 'exp': {'fc_var'},
 'randn_like': {'fc_var'},
 'mul_1': {'fc_var'},
 'add': {'fc_mu', 'fc_var'},
 'decoder_input': {'fc_mu', 'fc_var'},
 'view': {'decoder_input'},
 'decoder.0.0': {'decoder_input'},
 'decoder.0.1': {'decoder.0.0'},
 'decoder.0.2': {'decoder.0.1'},
 'decode